In [17]:
import os
import json
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

def load_and_process_data(directory):
    data = []
    labels_type = []
    labels_season = []
    
    for file in os.listdir(directory):
        if file.endswith('.json'):
            with open(os.path.join(directory, file), 'r', encoding='utf-8') as f:
                json_data = json.load(f)
                
                gender = json_data['metadata.model']['metadata.model.gender']
                age = json_data['metadata.model']['metadata.model.age']
                clothes_type = json_data['metadata.clothes']['clothing_type']
                clothes_season = json_data['metadata.clothes']['metadata.clothes.season']
                
                # 나이를 숫자로 변환
                if '10대' in age:
                    age = 15
                elif '20대' in age:
                    age = 25
                elif '30대' in age:
                    age = 35
                elif '40대' in age:
                    age = 45
                elif '50대' in age:
                    age = 55
                elif '60대' in age:
                    age = 65
                else:
                    age = 0  # 나이를 알 수 없거나 범위를 벗어난 경우
                
                # 성별을 숫자로 변환
                if gender == 'FEMALE':
                    gender = 0
                elif gender == 'MALE':
                    gender = 1
                else:
                    gender = -1  # 알 수 없는 성별
                
                # 온도 데이터를 season 키값을 참조하여 설정
                if clothes_season == 'summer':
                    temperature = 25 + np.random.rand() * 10  # 25도 이상
                elif clothes_season == 'winter':
                    temperature = 5 * np.random.rand()  # 5도 이하
                else:
                    temperature = 10 + np.random.rand() * 15  # spring 또는 fall
                
                data.append([gender, age, temperature])
                labels_type.append(clothes_type)
                labels_season.append(clothes_season)
    
    # 데이터프레임으로 변환
    df = pd.DataFrame(data, columns=['gender', 'age', 'temperature'])
    
    # 라벨 인코딩
    le_type = LabelEncoder()
    le_season = LabelEncoder()
    df['clothes_type'] = le_type.fit_transform(labels_type)
    df['clothes_season'] = le_season.fit_transform(labels_season)
    
    # One-hot 인코딩
    y_type = to_categorical(df['clothes_type'])
    y_season = to_categorical(df['clothes_season'])
    
    return df[['gender', 'age', 'temperature']].values, y_type, y_season, le_type, le_season

directory_path = r"Dataset\bottom"
X, y_type, y_season, le_type, le_season = load_and_process_data(directory_path)


In [33]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout

# 입력 데이터의 형태
input_shape = (3,)  # 성별, 나이, 온도

# 입력층
inputs = Input(shape=input_shape)

# 은닉층
x = Dense(128, activation='relu')(inputs)
x = Dropout(0.5)(x)  # 드롭아웃 층 추가
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)  # 드롭아웃 층 추가
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)  # 드롭아웃 층 추가
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)  # 드롭아웃 층 추가

# 출력층
output_type = Dense(y_type.shape[1], activation='softmax', name='type_output')(x)
output_season = Dense(y_season.shape[1], activation='softmax', name='season_output')(x)

# 모델 구성
model = Model(inputs=inputs, outputs=[output_type, output_season])

# 모델 컴파일
model.compile(optimizer='adam',
              loss={'type_output': 'categorical_crossentropy', 'season_output': 'categorical_crossentropy'},
              metrics={'type_output': 'accuracy', 'season_output': 'accuracy'})

# 모델 학습
model.fit(X, {'type_output': y_type, 'season_output': y_season}, epochs=20, batch_size=32)

# 모델 저장
model.save('clothing_model.h5')

# 모델 평가
results = model.evaluate(X, {'type_output': y_type, 'season_output': y_season})

# 평가 결과를 출력
print(results)


Epoch 1/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 923us/step - loss: 6.1412 - season_output_accuracy: 0.3540 - type_output_accuracy: 0.4238
Epoch 2/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 960us/step - loss: 2.4260 - season_output_accuracy: 0.4414 - type_output_accuracy: 0.5300
Epoch 3/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 984us/step - loss: 2.2199 - season_output_accuracy: 0.4898 - type_output_accuracy: 0.5980
Epoch 4/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 926us/step - loss: 2.1213 - season_output_accuracy: 0.4917 - type_output_accuracy: 0.6200
Epoch 5/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 965us/step - loss: 2.0622 - season_output_accuracy: 0.5305 - type_output_accuracy: 0.6355
Epoch 6/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 999us/step - loss: 1.9025 - season_output_accuracy: 0.5966 - type_output_accuracy: 0.6345
Epoch 7/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 961us/step - loss: 1.8250 - season_output_accuracy: 0.6402 - type_output_accuracy: 0.6546
Epoch 8/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.7583 - season_output_ac

80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 584us/step - loss: 1.3036 - season_output_accuracy: 0.7965 - type_output_accuracy: 0.6520 
[1.312765121459961, 0.797095775604248, 0.6605180501937866]


In [42]:
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder

# 라벨 인코더를 로드하기 위해 학습 데이터의 클래스들로 인코더 재생성
def create_label_encoders(type_classes, season_classes):
    le_type = LabelEncoder()
    le_season = LabelEncoder()
    le_type.fit(type_classes)
    le_season.fit(season_classes)
    return le_type, le_season

# 입력 데이터를 받아서 예측하는 함수
def predict_clothes_type_and_season(model, le_type, le_season, gender, age, temperature):
    # 입력 데이터를 넘파이 배열로 변환
    input_data = np.array([[gender, age, temperature]])
    
    # 예측 수행
    predictions = model.predict(input_data)
    
    # 예측 결과를 라벨로 변환
    predicted_type = np.argmax(predictions[0], axis=1)
    predicted_season = np.argmax(predictions[1], axis=1)
    
    predicted_type_label = le_type.inverse_transform(predicted_type)[0]
    predicted_season_label = le_season.inverse_transform(predicted_season)[0]
    
    return predicted_type_label, predicted_season_label

# 모델 로드
model_path = 'clothing_model.h5'
model = load_model(model_path)

# 학습 데이터로부터 라벨 인코더 생성 (여기서는 클래스 이름들을 수동으로 입력해야 합니다)
type_classes = ['숏스커트', '긴바지', '반바지', '롱스커트']  # 실제 데이터 클래스 이름들을 여기에 나열
season_classes = ['spring&fall', 'winter', 'summer']

le_type, le_season = create_label_encoders(type_classes, season_classes)

# 새로운 입력 데이터를 받아 예측
gender = 0  # FEMALE
age = 40  # 20대
temperature = 30 # 예시 온도

predicted_type, predicted_season = predict_clothes_type_and_season(model, le_type, le_season, gender, age, temperature)

print(f"Predicted Type: {predicted_type}")
print(f"Predicted Season: {predicted_season}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Predicted Type: 긴바지
Predicted Season: spring&fall


In [12]:
%pip install pydot